In [3]:
import requests
import random
import pandas as pd
import json
from ipywidgets import IntProgress
from IPython.display import display

# get tags for subset `quant.csv`

In [4]:
quant = pd.read_csv("../data/02_subset_images/quant.csv")

In [ ]:
# track progress

images_random = {}
for image in quant.name:
    images_random[image] = False

In [ ]:
# select 10 random tags for each image from the subset
# check that "amd64" is listed among the architecture of the tag

counter = 0
N = 10

for image in quant.name:
    print(f"Getting {image}")
    if images_random[image] == False:
        tags = []
        try:
            pref, suf = image.split("/")
            link = f"https://hub.docker.com/v2/namespaces/{pref}/repositories/{suf}/tags"
            res = requests.get(link)
            res.raise_for_status()
            res = res.json()
            count = res["count"]
            up = count if count < 10 else count // 2
            iteration = 0
            total = 0
            while total < N and iteration < 100:
                page_num = random.randint(1, up)
                tag = requests.get(link, params={"page": page_num, "page_size": 1}).json()["results"][0]
                if "amd64" in [i["architecture"] for i in tag["images"]]:
                    tags.append(tag)
                    total += 1
                iteration += 1
            
            if iteration == 100:
                print("Error: not enough amd64 images found")
            with open(f"../data/03_list_tags/res/{pref}_{suf}.json", "w") as f:
                json.dump(tags, f, ensure_ascii=False, indent=4)
        except:
            print(f"Error with {image}")
        

    images_random[image] = True
    counter += 1
    percent = round((counter / len(images_random)) * 100)
    print("x" * percent, "_" * (100 - percent), f" {counter} of {len(images_random)} ({percent}%)", sep="")

# leave tags for the smaller subset of images `quant_1000.csv`

In [5]:
quant_1000 = pd.read_csv("../data/02_subset_images/quant_1000.csv")

In [ ]:
# list of tags
# here about 3 tags per image are selected

with open("../data/03_list_tags/tags_flat_3000.json", "r") as f:
    tags_flat_list = json.load(f)

In [ ]:
images = {}
for image in quant_1000.name:
    images[image] = False

In [ ]:
tags = {}

for image in quant_1000.name:
    tags_for_image = {}
    pref, suf = image.split("/")
    with open(f"../data/03_list_tags/res/{pref}_{suf}.json", "r") as f:
        tags_from_file = json.load(f)
        
    # check tags from the list
    matching_tags = [q for q in tags_flat_list if f"{pref}/{suf}" in q]
    for tag in matching_tags:
        pref, suf, postf = tag.split("_", 2)
        matching_tag_from_file = [q for q in tags_from_file if postf == q.get("name")]
        if len(matching_tag_from_file) == 1:
            matching_tag = matching_tag_from_file[0]
            if "linux" in [q.get("os") for q in matching_tag.get("images", [])]:
                tags_for_image[f"{image}:{matching_tag['name']}"] = matching_tag["last_updated"]
                print(f"{image}:{matching_tag['name']}")
                if len(tags_for_image) > 2:
                    break
    
    i = 0
    while i < len(tags_from_file) and len(tags_for_image) < 3:
        if "linux" in [q.get("os") for q in tags_from_file[i].get("images", [])]:
            tags_for_image[f"{image}:{tags_from_file[i]['name']}"] = tags_from_file[i]["last_updated"]
        i += 1
    tags[image] = tags_for_image

In [ ]:
with open("tags/tags_random_amd64/tags_3000.json", "w") as f:
    json.dump(tags, f)